In [19]:
import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime
import plotly.express as px

In [45]:
#import datetime
#init = datetime.datetime.now()

# Load csv dataset
data_train = pd.read_csv("/Users/farahboukitab/code/mrdaraujo/business_case_869/business_case_869/data/store-sales-time-series-forecasting/train.csv")
data_stores = pd.read_csv("/Users/farahboukitab/code/mrdaraujo/business_case_869/business_case_869/data/store-sales-time-series-forecasting/stores.csv")
df_heatmap = pd.read_csv("/Users/farahboukitab/code/mrdaraujo/business_case_869/business_case_869/data/store-sales-time-series-forecasting/Heatmap.csv")


# 1 ----- Dataframe to merge train dataset and stores dataset
data_train_merge_stores = pd.merge(data_train, data_stores, on="store_nbr")
data_train_merge_stores['date'] = pd.to_datetime(data_train_merge_stores['date'])
data_train_merge_stores['month'] = data_train_merge_stores['date'].dt.month
data_train_merge_stores['year'] = data_train_merge_stores['date'].dt.year
data_train_merge_stores = data_train_merge_stores.drop(data_train_merge_stores[data_train_merge_stores['year']==2017].index, axis=0, inplace=False)
data_train_merge_stores = data_train_merge_stores.drop(['id','onpromotion','state','type','cluster'], axis=1, inplace=False)
#save to csv
data_train_merge_stores.to_csv('/Users/farahboukitab/code/mrdaraujo/business_case_869/business_case_869/data/store-sales-time-series-forecasting/data_train_merge_stores.csv', index=False) 


# 2 ----- Dataframe to build the map and city
df_heatmap.rename(columns = {'Unnamed: 0':'city', 'Unnamed: 1':'Lat', 'Unnamed: 2':'Lon', 'Unnamed: 3':'Weight'}, inplace = True)
df_heatmap.drop([0], axis=0, inplace=True)
#save to csv
df_heatmap.to_csv('/Users/farahboukitab/code/mrdaraujo/business_case_869/business_case_869/data/store-sales-time-series-forecasting/df_heatmap.csv', index=False) 

#3 ----- Dataframe with merge train dataset and stores dataset with heatmap
map_base = pd.merge(data_train_merge_stores, df_heatmap, on='city').drop(columns=['Weight', 'type', 'cluster', 'state', 'onpromotion'])
map_base['date'] = pd.to_datetime(map_base['date'])
map_base['month'] = map_base['date'].dt.month
map_base['year'] = map_base['date'].dt.year
map_base['Lat'] = pd.to_numeric(map_base['Lat'])
map_base['Lon'] = pd.to_numeric(map_base['Lon'])
#save to csv
map_base.to_csv('/Users/farahboukitab/code/mrdaraujo/business_case_869/business_case_869/data/store-sales-time-series-forecasting/map_base.csv', index=False) 

#4 ----- Dataframe for the map with city, lat, lon and sales
sales_city_year = pd.DataFrame(map_base.groupby(['city', 'Lat', 'Lon'])['sales'].sum()).reset_index().sort_values(by='sales', ascending=False)
#save to csv
sales_city_year.to_csv('/Users/farahboukitab/code/mrdaraujo/business_case_869/business_case_869/data/store-sales-time-series-forecasting/sales_city_year.csv', index=False) 


#5 ----- Dataframe with top 5 cities
map_base_top_five = map_base[map_base['city'].isin(['Quito', 'Guayaquil', 'Cuenca', 'Ambato', 'Santo Domingo'])]
map_base_top_five = pd.DataFrame(map_base_top_five.groupby(['city', 'year'])['sales'].sum()).reset_index().sort_values(by=[ 'year','sales', 'city' ], ascending=[True,False , True])
map_base_top_five['year'] = map_base_top_five['year'].astype('string')
#save to csv
map_base_top_five.to_csv('/Users/farahboukitab/code/mrdaraujo/business_case_869/business_case_869/data/store-sales-time-series-forecasting/map_base_top_five.csv', index=False) 


#6 ----- Dataframe with cities and number of stores in each city
df_stores_city = pd.DataFrame(data_stores.groupby(['city'])['store_nbr'].count())
df_stores_city = df_stores_city.reset_index().sort_values(by='store_nbr', ascending=False)
#save to csv
df_stores_city.to_csv('/Users/farahboukitab/code/mrdaraujo/business_case_869/business_case_869/data/store-sales-time-series-forecasting/df_stores_city.csv', index=False) 


#7 ------Dataframe with the store that sales most of sales in each city
df_MaxSales_BiggestStore = pd.DataFrame(data_train_merge_stores.groupby(['city','store_nbr'])['sales'].sum()).reset_index()
df_MaxSales_BiggestStore = df_MaxSales_BiggestStore.sort_values(by = ["city", "sales"])
df_MaxSales_BiggestStore = df_MaxSales_BiggestStore.drop_duplicates( subset = ["city"], keep='last').sort_values('sales',ascending = False)
df_MaxSales_BiggestStore['store_nbr'] = df_MaxSales_BiggestStore['store_nbr'].astype('string')
#save to csv
df_MaxSales_BiggestStore.to_csv('/Users/farahboukitab/code/mrdaraujo/business_case_869/business_case_869/data/store-sales-time-series-forecasting/df_MaxSales_BiggestStore.csv', index=False) 

#8 ----- Dataframe with the top 5 stores
df_stores_top_five = df_MaxSales_BiggestStore.head(5)
#save to csv
df_stores_top_five.to_csv('/Users/farahboukitab/code/mrdaraujo/business_case_869/business_case_869/data/store-sales-time-series-forecasting/df_stores_top_five.csv', index=False) 

#9 ----- Dataframe for family per year
data_family_peryear = data_train_merge_stores.groupby(["year","month","family","store_nbr", "date"]).sum().reset_index().drop(columns=['cluster'])
data_family_peryear = pd.DataFrame(data_family_peryear.groupby(['year','family'])['sales'].sum()).reset_index()
#save to csv
data_family_peryear.to_csv('/Users/farahboukitab/code/mrdaraujo/business_case_869/business_case_869/data/store-sales-time-series-forecasting/data_family_peryear.csv', index=False)

#10 ----- Dataframe for family total year
data_family_allyear = pd.DataFrame(data_train_merge_stores.groupby(['family'])['sales'].sum().reset_index()).sort_values(by='sales', ascending=False)
data_family_allyear = data_family_allyear.sort_values(by='sales', ascending=False)
#save to csv
data_family_allyear.to_csv('/Users/farahboukitab/code/mrdaraujo/business_case_869/business_case_869/data/store-sales-time-series-forecasting/data_family_allyear.csv', index=False)

#11 ------ Dataframe with top 5 families total year
data_top5family = data_family_allyear.head(5)
#save to csv
data_top5family.to_csv('/Users/farahboukitab/code/mrdaraujo/business_case_869/business_case_869/data/store-sales-time-series-forecasting/data_top5family.csv', index=False)

#finish = datetime.datetime.now()
#print(finish-init)

0:00:06.776363


In [20]:
data_train_merge_stores = pd.read_csv("/Users/farahboukitab/code/mrdaraujo/business_case_869/business_case_869/data/store-sales-time-series-forecasting/data_train_merge_stores.csv")

In [21]:
data_train_merge_stores

,date,store_nbr,family,sales,city,month,year
0,2013-01-01,1,AUTOMOTIVE,0.000,Quito,1,2013
1,2013-01-01,1,BABY CARE,0.000,Quito,1,2013
2,2013-01-01,1,BEAUTY,0.000,Quito,1,2013
3,2013-01-01,1,BEVERAGES,0.000,Quito,1,2013
4,2013-01-01,1,BOOKS,0.000,Quito,1,2013
...,...,...,...,...,...,...,...
2596369,2016-12-31,9,POULTRY,687.853,Quito,12,2016
2596370,2016-12-31,9,PREPARED FOODS,100.405,Quito,12,2016
2596371,2016-12-31,9,PRODUCE,3091.356,Quito,12,2016
2596372,2016-12-31,9,SCHOOL AND OFFICE SUPPLIES,2.000,Quito,12,2016
